In [1]:
import pandas as pd
import glob
import os
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar todos os arquivos CSV de uma pasta específica em um único DataFrame
folder_path = 'csvs'  # Substitua pelo caminho da sua pasta
csv_files = glob.glob(folder_path + '/*.csv')

# Definir as colunas a serem importadas e seus tipos de dados
usecols = [
    'hour',
    'total_trip_count',
    'forward_commute_trip_count', 'reverse_commute_trip_count',
    'forward_leisure_trip_count', 'reverse_leisure_trip_count',
    'forward_male_people_count', 'reverse_male_people_count',
    'forward_female_people_count', 'reverse_female_people_count',
    'forward_unspecified_people_count', 'reverse_unspecified_people_count',
    'forward_18_34_people_count', 'reverse_18_34_people_count',
    'forward_35_54_people_count', 'reverse_35_54_people_count',
    'forward_55_64_people_count', 'reverse_55_64_people_count',
    'forward_65_plus_people_count', 'reverse_65_plus_people_count'
]

dtype_dict = {
    'total_trip_count': 'Int16',
    'forward_commute_trip_count': 'Int16', 'reverse_commute_trip_count': 'Int16',
    'forward_leisure_trip_count': 'Int16', 'reverse_leisure_trip_count': 'Int16',
    'forward_male_people_count': 'Int16', 'reverse_male_people_count': 'Int16',
    'forward_female_people_count': 'Int16', 'reverse_female_people_count': 'Int16',
    'forward_unspecified_people_count': 'Int16', 'reverse_unspecified_people_count': 'Int16',
    'forward_18_34_people_count': 'Int16', 'reverse_18_34_people_count': 'Int16',
    'forward_35_54_people_count': 'Int16', 'reverse_35_54_people_count': 'Int16',
    'forward_55_64_people_count': 'Int16', 'reverse_55_64_people_count': 'Int16',
    'forward_65_plus_people_count': 'Int16', 'reverse_65_plus_people_count': 'Int16'
}

# Carregar todos os CSVs em uma lista e concatenar em um único DataFrame
df_list = [pd.read_csv(file, usecols=usecols, dtype=dtype_dict, parse_dates=['hour']) for file in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# Extrair ano, mês, dia da semana e hora do dia
df_all['year'] = df_all['hour'].dt.year
df_all['month'] = df_all['hour'].dt.month
df_all['day_of_week'] = df_all['hour'].dt.dayofweek  # 0 = Segunda-feira, 6 = Domingo
df_all['hour_of_day'] = df_all['hour'].dt.hour

# Calcular os totais para gêneros e faixas etárias fora da função
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Mapear números dos dias da semana para nomes em português
day_names = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

# Criar widgets conforme solicitado
activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano_widget = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

mes_widget = widgets.SelectMultiple(
    options=[('Janeiro', 1), ('Fevereiro', 2), ('Março', 3), ('Abril', 4), ('Maio', 5), ('Junho', 6),
             ('Julho', 7), ('Agosto', 8), ('Setembro', 9), ('Outubro', 10), ('Novembro', 11), ('Dezembro', 12)],
    value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Mês:'
)

genero_widget = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria_widget = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

# Mapas para mapeamento das opções dos widgets para valores no DataFrame
map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

# Definir a função para atualizar o gráfico
def update_plot(activity_type, ano, mes, genero, faixa_etaria):
    # Mapear as opções selecionadas para os valores correspondentes
    activity_type_mapped = map_atividade[activity_type]
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]

    # Filtrar dados pelos anos e meses selecionados
    df_filtered = df_all[(df_all['year'].isin(ano)) & (df_all['month'].isin(mes))]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    # Selecionar a coluna de contagem de viagens apropriada
    if activity_type_mapped == 'total':
        trip_count_column = 'total_trip_count'
    else:
        trip_count_column = activity_type_mapped + '_trip_count'  # 'commute_trip_count' ou 'leisure_trip_count'

    # Tratar a seleção de gênero
    if genero_mapped != 'total':
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['people_count'] = df_filtered[['male_people_count', 'female_people_count', 'unspecified_people_count']].fillna(0).sum(axis=1)

    # Tratar a seleção de faixa etária
    if faixa_etaria_mapped != 'total':
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].fillna(0).sum(axis=1)

    # Filtrar dados onde as contagens de pessoas são maiores que zero
    df_filtered = df_filtered[(df_filtered['people_count'] > 0) & (df_filtered['age_group_count'] > 0)]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    # Agrupar os dados por 'day_of_week' e somar as contagens de viagens
    grouped = df_filtered.groupby('day_of_week')[trip_count_column].sum().reset_index()

    # Mapear números dos dias da semana para nomes em português
    grouped['day_name'] = grouped['day_of_week'].map(day_names)

    # Ordenar os dados por 'day_of_week'
    grouped.sort_values('day_of_week', inplace=True)

    # Plotar os dados
    plt.figure(figsize=(10, 6))
    sns.barplot(x='day_name', y=trip_count_column, data=grouped, color='skyblue')
    plt.title(f"Viagens por Dia da Semana ({min(ano)} a {max(ano)}) - Atividade: {activity_type} - Faixa Etária: {faixa_etaria}")
    plt.xlabel('Dia da Semana')
    plt.ylabel('Número de Viagens')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Determinar o valor do filtro para nomear o arquivo
    if faixa_etaria_mapped != 'total':
        valor_filtro = faixa_etaria_mapped
    elif genero_mapped != 'total':
        valor_filtro = genero_mapped
    else:
        valor_filtro = 'total'

    pasta = 'graficos-semana'
    
    if not os.path.exists(pasta):
        os.makedirs(pasta)

    nome_arquivo = os.path.join(pasta, f"grafico-{valor_filtro}-semana.png")
    
    # Salvar o gráfico como um arquivo PNG
    plt.savefig(nome_arquivo)
    plt.show()

# Tornar a função interativa com os widgets
interact(
    update_plot,
    activity_type=activity_type_widget,
    ano=ano_widget,
    mes=mes_widget,
    genero=genero_widget,
    faixa_etaria=faixa_etaria_widget
)


interactive(children=(Dropdown(description='Tipo de Atividade:', options=('Todos', 'Deslocamento', 'Lazer'), v…

<function __main__.update_plot(activity_type, ano, mes, genero, faixa_etaria)>

Grafico de linha

In [ ]:
import pandas as pd
import glob
import os
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar todos os arquivos CSV de uma pasta específica em um único DataFrame
folder_path = 'csvs'  # Substitua pelo caminho da sua pasta
csv_files = glob.glob(folder_path + '/*.csv')

# Definir as colunas a serem importadas e seus tipos de dados
usecols = [
    'hour',
    'total_trip_count',
    'forward_commute_trip_count', 'reverse_commute_trip_count',
    'forward_leisure_trip_count', 'reverse_leisure_trip_count',
    'forward_male_people_count', 'reverse_male_people_count',
    'forward_female_people_count', 'reverse_female_people_count',
    'forward_unspecified_people_count', 'reverse_unspecified_people_count',
    'forward_18_34_people_count', 'reverse_18_34_people_count',
    'forward_35_54_people_count', 'reverse_35_54_people_count',
    'forward_55_64_people_count', 'reverse_55_64_people_count',
    'forward_65_plus_people_count', 'reverse_65_plus_people_count'
]

dtype_dict = {
    'total_trip_count': 'Int16',
    'forward_commute_trip_count': 'Int16', 'reverse_commute_trip_count': 'Int16',
    'forward_leisure_trip_count': 'Int16', 'reverse_leisure_trip_count': 'Int16',
    'forward_male_people_count': 'Int16', 'reverse_male_people_count': 'Int16',
    'forward_female_people_count': 'Int16', 'reverse_female_people_count': 'Int16',
    'forward_unspecified_people_count': 'Int16', 'reverse_unspecified_people_count': 'Int16',
    'forward_18_34_people_count': 'Int16', 'reverse_18_34_people_count': 'Int16',
    'forward_35_54_people_count': 'Int16', 'reverse_35_54_people_count': 'Int16',
    'forward_55_64_people_count': 'Int16', 'reverse_55_64_people_count': 'Int16',
    'forward_65_plus_people_count': 'Int16', 'reverse_65_plus_people_count': 'Int16'
}

# Carregar todos os CSVs em uma lista e concatenar em um único DataFrame
df_list = [pd.read_csv(file, usecols=usecols, dtype=dtype_dict, parse_dates=['hour']) for file in csv_files]
df_all = pd.concat(df_list, ignore_index=True)

# Extrair ano, mês, dia da semana e hora do dia
df_all['year'] = df_all['hour'].dt.year
df_all['month'] = df_all['hour'].dt.month
df_all['day_of_week'] = df_all['hour'].dt.dayofweek  # 0 = Segunda-feira, 6 = Domingo
df_all['hour_of_day'] = df_all['hour'].dt.hour

# Calcular os totais para gêneros e faixas etárias fora da função
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Mapear números dos dias da semana para nomes em português
day_names = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

# Criar widgets conforme solicitado
activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano_widget = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

mes_widget = widgets.SelectMultiple(
    options=[('Janeiro', 1), ('Fevereiro', 2), ('Março', 3), ('Abril', 4), ('Maio', 5), ('Junho', 6),
             ('Julho', 7), ('Agosto', 8), ('Setembro', 9), ('Outubro', 10), ('Novembro', 11), ('Dezembro', 12)],
    value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Mês:'
)

genero_widget = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria_widget = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

# Mapas para mapeamento das opções dos widgets para valores no DataFrame
map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

# Definir a função para atualizar o gráfico
def update_plot(activity_type, ano, mes, genero, faixa_etaria):
    # Mapear as opções selecionadas para os valores correspondentes
    activity_type_mapped = map_atividade[activity_type]
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]

    # Filtrar dados pelos anos e meses selecionados
    df_filtered = df_all[(df_all['year'].isin(ano)) & (df_all['month'].isin(mes))]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    # Selecionar a coluna de contagem de viagens apropriada
    if activity_type_mapped == 'total':
        trip_count_column = 'total_trip_count'
    else:
        trip_count_column = activity_type_mapped + '_trip_count'  # 'commute_trip_count' ou 'leisure_trip_count'

    # Tratar a seleção de gênero
    if genero_mapped != 'total':
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['people_count'] = df_filtered[['male_people_count', 'female_people_count', 'unspecified_people_count']].fillna(0).sum(axis=1)

    # Tratar a seleção de faixa etária
    if faixa_etaria_mapped != 'total':
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].fillna(0).sum(axis=1)

    # Filtrar dados onde as contagens de pessoas são maiores que zero
    df_filtered = df_filtered[(df_filtered['people_count'] > 0) & (df_filtered['age_group_count'] > 0)]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    plt.figure(figsize=(10, 6))

    if faixa_etaria_mapped == 'total':
        # Reorganizar os dados para ter uma coluna de faixa etária
        age_group_columns = ['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']
        df_melted = df_filtered.melt(
            id_vars=['day_of_week', trip_count_column],
            value_vars=age_group_columns,
            var_name='age_group',
            value_name='age_group_count_melted'
        )
        # Mapear os valores da coluna 'age_group' para rótulos legíveis
        df_melted['age_group'] = df_melted['age_group'].map({
            '18_34_people_count': '18-34',
            '35_54_people_count': '35-54',
            '55_64_people_count': '55-64',
            '65_plus_people_count': '65+'
        })

        df_melted = df_melted[df_melted['age_group_count_melted'] > 0]
        if df_melted.empty:
            print("Não há dados disponíveis para os filtros selecionados.")
            return
        # Agrupar os dados por 'day_of_week' e 'age_group'
        grouped = df_melted.groupby(['day_of_week', 'age_group'])[trip_count_column].sum().reset_index()
        # Mapear números dos dias da semana para nomes em português
        grouped['day_name'] = grouped['day_of_week'].map(day_names)
        sns.lineplot(x='day_name', y=trip_count_column, hue='age_group', data=grouped, marker='o')
    else:
        # Agrupar os dados por 'day_of_week' e somar as contagens de viagens
        grouped = df_filtered.groupby('day_of_week')[trip_count_column].sum().reset_index()
        grouped['day_name'] = grouped['day_of_week'].map(day_names)
        sns.barplot(x='day_name', y=trip_count_column, data=grouped, color='skyblue')

    plt.title(f"Viagens por Dia da Semana ({min(ano)} a {max(ano)}) - Atividade: {activity_type} - Faixa Etária: {faixa_etaria}")
    plt.xlabel('Dia da Semana')
    plt.ylabel('Número de Viagens')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Salvar o gráfico
    if faixa_etaria_mapped != 'total':
        valor_filtro = faixa_etaria_mapped
    elif genero_mapped != 'total':
        valor_filtro = genero_mapped
    else:
        valor_filtro = 'total'

    pasta = 'graficos-semana'

    if not os.path.exists(pasta):
        os.makedirs(pasta)

    nome_arquivo = os.path.join(pasta, f"grafico-{valor_filtro}-semana.png")
    plt.savefig(nome_arquivo)
    plt.show()

# Tornar a função interativa com os widgets
interact(
    update_plot,
    activity_type=activity_type_widget,
    ano=ano_widget,
    mes=mes_widget,
    genero=genero_widget,
    faixa_etaria=faixa_etaria_widget
)


interactive(children=(Dropdown(description='Tipo de Atividade:', options=('Todos', 'Deslocamento', 'Lazer'), v…

<function __main__.update_plot(activity_type, ano, mes, genero, faixa_etaria)>

Grafico barras Generos

In [ ]:
import pandas as pd
import glob
import os
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar todos os arquivos CSV de uma pasta específica em um único DataFrame
folder_path = 'csvs'  # Substitua pelo caminho da sua pasta
csv_files = glob.glob(folder_path + '/*.csv')

# Definir as colunas a serem importadas e seus tipos de dados
usecols = [
    'hour',
    'total_trip_count',
    'forward_commute_trip_count', 'reverse_commute_trip_count',
    'forward_leisure_trip_count', 'reverse_leisure_trip_count',
    'forward_male_people_count', 'reverse_male_people_count',
    'forward_female_people_count', 'reverse_female_people_count',
    'forward_unspecified_people_count', 'reverse_unspecified_people_count',
    'forward_18_34_people_count', 'reverse_18_34_people_count',
    'forward_35_54_people_count', 'reverse_35_54_people_count',
    'forward_55_64_people_count', 'reverse_55_64_people_count',
    'forward_65_plus_people_count', 'reverse_65_plus_people_count'
]

dtype_dict = {
    'total_trip_count': 'Int16',
    'forward_commute_trip_count': 'Int16', 'reverse_commute_trip_count': 'Int16',
    'forward_leisure_trip_count': 'Int16', 'reverse_leisure_trip_count': 'Int16',
    'forward_male_people_count': 'Int16', 'reverse_male_people_count': 'Int16',
    'forward_female_people_count': 'Int16', 'reverse_female_people_count': 'Int16',
    'forward_unspecified_people_count': 'Int16', 'reverse_unspecified_people_count': 'Int16',
    'forward_18_34_people_count': 'Int16', 'reverse_18_34_people_count': 'Int16',
    'forward_35_54_people_count': 'Int16', 'reverse_35_54_people_count': 'Int16',
    'forward_55_64_people_count': 'Int16', 'reverse_55_64_people_count': 'Int16',
    'forward_65_plus_people_count': 'Int16', 'reverse_65_plus_people_count': 'Int16'
}

# Carregar todos os CSVs em uma lista e concatenar em um único DataFrame
df_list = [pd.read_csv(
    file,
    usecols=usecols,
    dtype=dtype_dict,
    parse_dates=['hour']
) for file in csv_files]

df_all = pd.concat(df_list, ignore_index=True)

# Extrair o ano, mês, dia da semana e hora do dia
df_all['year'] = df_all['hour'].dt.year
df_all['month'] = df_all['hour'].dt.month
df_all['day_of_week'] = df_all['hour'].dt.dayofweek  # Segunda=0, Domingo=6
df_all['hour_of_day'] = df_all['hour'].dt.hour

# Calcular os totais para gêneros e faixas etárias
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Mapear números dos dias da semana para nomes em português
day_names = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

# Criar widgets conforme solicitado
activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano_widget = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

mes_widget = widgets.SelectMultiple(
    options=[('Janeiro', 1), ('Fevereiro', 2), ('Março', 3), ('Abril', 4), ('Maio', 5), ('Junho', 6),
             ('Julho', 7), ('Agosto', 8), ('Setembro', 9), ('Outubro', 10), ('Novembro', 11), ('Dezembro', 12)],
    value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Mês:'
)

genero_widget = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria_widget = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

# Mapas para mapeamento das opções dos widgets para valores no DataFrame
map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

# Definir a função para atualizar o gráfico
def update_plot(activity_type, ano, mes, genero, faixa_etaria):
    # Mapear as opções selecionadas para os valores correspondentes
    activity_type_mapped = map_atividade[activity_type]
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]

    # Filtrar dados pelos anos e meses selecionados
    df_filtered = df_all[(df_all['year'].isin(ano)) & (df_all['month'].isin(mes))]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    # Selecionar a coluna de contagem de viagens apropriada
    if activity_type_mapped == 'total':
        trip_count_column = 'total_trip_count'
    else:
        trip_count_column = activity_type_mapped + '_trip_count'  # 'commute_trip_count' ou 'leisure_trip_count'

    # Tratar a seleção de faixa etária
    if faixa_etaria_mapped != 'total':
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)
    else:
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].fillna(0).sum(axis=1)

    # Filtrar dados onde as contagens de faixa etária são maiores que zero
    df_filtered = df_filtered[df_filtered['age_group_count'] > 0]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return  # Sair da função se não houver dados após o filtro

    plt.figure(figsize=(10, 6))

    # Tratar a seleção de gênero
    if genero_mapped != 'total':
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)
        df_filtered = df_filtered[df_filtered['people_count'] > 0]
        if df_filtered.empty:
            print("Não há dados disponíveis para os filtros selecionados.")
            return
        valor_filtro = genero_mapped
        # Agrupar os dados por 'day_of_week' e somar as contagens de viagens
        grouped = df_filtered.groupby('day_of_week')[trip_count_column].sum().reset_index()
        grouped['day_name'] = grouped['day_of_week'].map(day_names)

        # Plotar os dados
        sns.barplot(x='day_name', y=trip_count_column, data=grouped)
        
    else:
        # Reorganizar os dados para ter uma coluna 'gender' e uma coluna 'people_count'
        gender_columns = ['male_people_count', 'female_people_count', 'unspecified_people_count']
        df_melted = df_filtered.melt(
            id_vars=['day_of_week', trip_count_column],
            value_vars=gender_columns,
            var_name='gender',
            value_name='people_count'
        )
        # Mapear os nomes das colunas para os gêneros correspondentes
        df_melted['gender'] = df_melted['gender'].map({
            'male_people_count': 'Homem',
            'female_people_count': 'Mulher',
            'unspecified_people_count': 'Não especificado'
        })

        # Filtrar dados onde as contagens de pessoas são maiores que zero
        df_melted = df_melted[df_melted['people_count'] > 0]

        if df_melted.empty:
            print("Não há dados disponíveis para os filtros selecionados.")
            return  # Sair da função se não houver dados após o filtro

        # Agrupar os dados por 'day_of_week' e 'gender' e somar as contagens de viagens
        grouped = df_melted.groupby(['day_of_week', 'gender'])[trip_count_column].sum().reset_index()
        grouped['day_name'] = grouped['day_of_week'].map(day_names)
        valor_filtro = 'todos_generos'

        # Plotar os dados com a distinção de gênero
        sns.barplot(x='day_name', y=trip_count_column, hue='gender', data=grouped)

    plt.title(f"Viagens por Dia da Semana ({min(ano)} a {max(ano)}) - Atividade: {activity_type}")
    plt.xlabel('Dia da Semana')
    plt.ylabel('Número de Viagens')
    plt.xticks(rotation=45)
    plt.legend(title='Gênero')
    plt.tight_layout()
    
    # Salvando o gráfico
    pasta = 'graficos-semana'
    if not os.path.exists(pasta):
        os.makedirs(pasta)
    nome_arquivo = os.path.join(pasta, f"grafico-{valor_filtro}-semana.png")
    plt.savefig(nome_arquivo)
    plt.show()

# Tornar a função interativa com os widgets
interact(
    update_plot,
    activity_type=activity_type_widget,
    ano=ano_widget,
    mes=mes_widget,
    genero=genero_widget,
    faixa_etaria=faixa_etaria_widget
)


interactive(children=(Dropdown(description='Tipo de Atividade:', options=('Todos', 'Deslocamento', 'Lazer'), v…

<function __main__.update_plot(activity_type, ano, mes, genero, faixa_etaria)>